# Homework 2

### 1. My similarity metric function has two steps. First, it reduces the information for each active site. For each residue I classify it into a category based on whether the residue is nonpolar, amphipathic, polar, or charged. I then take the two  simplified strings of residues and align the active sites in a pairwise fashion using a global alignment method which assigns + points for a correctly aligned base pair, and penalizes gaps and extensions in gaps. I used global pairwise as opposed to local because these active sites are already pruned of non-relevant residues, and so comparing the entire active sites together makes sense. I did some testing of the parameters to come up with the parameters used to assign correct or incorrect matches. After obtaining an alignment score I then calculated the maximum alignment possible alignment score for the two sequences, and divided the actual/max to get a percent alignment score. 
### I devised this scoring system based on the assumptions that 1) biologically amino acids can often be interchangable based on shared function and 2) active sites with similar sequences have similar function. The first assumption although it does not hold in every case, allows for a greater degree of "wiggle" room between alignments and is built into the first step. For example, valine and isoleucine are both nonpolar amino acids and often are interchangable functionally. By removing the distinction between them and categorizing them both as "n" for "nonpolar", I still retain the information of their functional assignments, but prevent the downstream sequence alignment algorithm from marking them as mismatches. The second assumption is that similar sequences -> similar functions, which is a pattern that has been validated fairly broadly in biology, so I believe that is a fair assumption as well. In summary then, my function seeks to quantify the degree of similarity measured between two active sites using: 1) a simplification of amino acids to functional class, to allow for mismatch tolerance between sequences based on functional identity and 2) a global pairwise alignment algorithm based on these simplified strings, and normalization by max possible score.



### 2. The partitioning algorithm I implemented is K-means. I chose to use this because K-means is more computationally efficient than some other methods, easy to implement, makes sense for our biological problem, and works well with large datasets. K-means uses centroids in the data, which works well with active site data. If we were instead to implement a medoid, average, or some other cluster center that is not actually an active site point we would have to somehow combine features of the active sites in that cluster. However, using an actual active site as the centroid simplifies that problem. K-means by centroids also makes sense biologically, assuming we have a large enough set of active sites, because if we were to estimate a novel cluster center based on the properties of the active sites in the cluster, we would essentially be creating a new 'meta-active site'. However, in the process of calculating this we might generate something that is very dissimilar from biology. By using the active site that has the greatest similarity to all the others in the cluster, we select the most representative active site in a class, and avoid the problem of potentially creating a fictional, non-biologically relevant active site. 

### 3. The hierarchical algorithm I chose to implement was single-linkage clustering (k-nearest neighbors). I chose this based on its ease of implementation, faster than divisive clustering, and applicability to our biological data set (active sites). While divisive clustering is O(2^n), single-linkage can be O(n^3) or O(n^2). Additionally, in comparison to a k-nearest neighbor implementation, the tree need only be calculated once and the clusters at each K = desired numbers of clusters can be inferred afterwards without further calculation. 
### The use of knn also made sense to me based on the problem at hand. Our dataset are active sites of proteins that have diverged throughout evolution. Whereas a divisive algorithm assumes that everything is in one cluster to start and breaks it up, using 'cluster similarity' as the metric, our Knn algorithm assumes that two proteins might be related and builds up from there. This method might catch proteins that are highly similar and build out cluster centers from there, allowing common themes in active sites to serve as the centroids around which to build clusters. Based on what we know about protein domains and active sites, there are reoccuring patterns (motifs) shared between proteins; by allowing Knn to nucleate clusters based around these shared motifs, we can apply our biological intution to the algorithm. 




In [1]:
from hw2skeleton import cluster
from hw2skeleton import io
%matplotlib inline
####import data
active_sites = io.read_active_sites('/Users/johnny/Desktop/class/hw2-skeleton/data')

Read in 136 active sites


In [ ]:
####partitioning evaluation
max_clust = 15
quality_scores = []
for k in range(1,max_clust+1):
    output = cluster.cluster_by_partitioning(active_sites,k)
    quality = cluster.calc_clust_dist(output)
    quality_scores.append(quality)
cluster.plt.violinplot(quality_scores)